In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [12]:
import cv2 #immporting OpenCV, numpy and time
import numpy as np
import time



net = cv2.dnn.readNet('yolov4-obj_1000.weights','yolov4-obj.cfg') #loading trained weight file and cfg file for helmet class.

classes = ['Helmet'] #because I am working on one class only



cap = cv2.VideoCapture(0) #using camera for detection


font = cv2.FONT_HERSHEY_DUPLEX 
starting_time = time.time() 
frame_id = 0 


while True:

    _, img = cap.read() #capturing frame from a live video and converting it into image and reading it
    height, width, _ = img.shape #getting shape of the image
    frame_id += 1   
    #now for yolo we have to resize the image in a square that fits the size for yolov3 or yolov4
    #also we need to normalize it by dividing the pixel value by 255 and the valued are also need to be in RGB order (it is in BGR so we need to swapping).
    #so for that preparing image called blob       
    blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = True, crop = False)
    #showing what we have done using blob from image
    #for b in blob:
    #   for n, img_blob in enumerate(b):
    #   cv2.imshow(str(n),img_blob)#this is showing our input image after the mean substraction normalizing and also the channel swapping 
    #so as in the output we can see the red channel, green channel and also the blue channel
    #this blob function is used to create for input image and to use it as an input to our models 
    #and taking blob image in input
    net.setInput(blob)
    #now we want to know bonding boxes or predicted classes and also need to get the output layera names and then pass the name into the forward function
    #creating for getting the output layer name first
    output_layers_names = net.getUnconnectedOutLayersNames()
    layersOutputs = net.forward(output_layers_names)
     #we are not ready yet, we need to extract the bounding boxes and the confidences and the predicted classes which will be started in two different lists
     #1st for boxes, 2nd for confidences and other also for 
    boxes = []
    confidences = []
    class_ids = []
    #creating bounding boxes
    for output in layersOutputs:   
        for detection in output:   #for extracting the information from each of the output
            scores = detection[5:]  #started for 6 , all the 18 classes predictions
            class_id = np.argmax(scores)  #extracting highest scores
            confidence = scores[class_id]  #for finding the maximum value from these class
            # a confidence that is high enough for us to consider that the object is being detected        

            if confidence > 0.5:
                center_x = int(detection[0]*width) #under the detection all it is normalized
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h]) #appending the boxes in thel ist and also 4 coordinates is here bcz it's a rectangular shape
                class_ids.append(class_id)
                confidences.append((float(confidence)))

    #print(len(boxes))
    #print(detection)
#now we will get multiple box for a single object so to only keep the higher scores we will use non maximum supressions 
#in a argument we will use boxes that we have got, the confiden
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes.flatten())
    #print(type(indexes))
        
    colors = np.random.uniform(0, 255, size = (len(boxes), 3)) 

    for i in range(len(boxes)):
        if i in indexes:
            x, y , w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 8)
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 1,(255,255,255) , 2)

    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(img, "FPS: " + str(fps), (10,30), font, 1, (0,0,0), 1)
    cv2.imshow('Output',cv2.resize(img,(700, 500)))  
    cv2.resize(img,(600, 400))
    key = cv2.waitKey(1)  #0
        
    #if the 'c' key is pressed, stop the loop
    if key == ord('c'):
       break    

        


cap.release()    
cv2.destroyAllWindows()